In [13]:
import pandas as pd

columns = ['birthDate',
           'nationality',
           'id',
           'jerseyNumber',
           'code',
           'type', 
           'abbreviation',
           'birthStateProvince',
           'Team_Number',
           'Unnamed: 0',
           'index',
           'alternateCaptain',
           'captain',
           'active',
           'rookie',
           'rosterStatus',
           'birthCity']

df = pd.read_csv('~/Desktop/NHL-Salary-Predictions/data/cleaned_player_df_dash.csv').drop(columns, axis=1)

# --------------------------- Player Salaries -------------------------------------------------

df = df[df['Salary_2020-21'] != 0.0]
df_datatable_1 = df[['fullName',
                     'Salary_2020-21',
                     'name',
                     'currentAge',
                     'height',
                     'weight',
                     'shootsCatches',
                     'birthCountry']] \
                    .sort_values('Salary_2020-21', ascending=False)
df_datatable_1['shootsCatches'] = df_datatable_1['shootsCatches'].replace('L', 'Left').replace('R', 'Right')
df_datatable_1['Rank'] = range(1, len(df_datatable_1) + 1)
df_datatable_1 = df_datatable_1[['Rank',
                                   'fullName',
                                   'Salary_2020-21',
                                   'name',
                                   'currentAge',
                                   'height',
                                   'weight',
                                   'shootsCatches',
                                   'birthCountry']]
df_datatable_1['id'] = df['fullName']
df_datatable_1.set_index('id', inplace=True, drop=False)

In [44]:
offense.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 476 entries, 0 to 475
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   fullName     476 non-null    object 
 1   timeOnIce20  427 non-null    object 
 2   assists20    427 non-null    float64
 3   goals20      427 non-null    float64
 4   pim20        427 non-null    float64
 5   shots20      427 non-null    float64
 6   games20      427 non-null    float64
 7   hits20       427 non-null    float64
dtypes: float64(6), object(2)
memory usage: 49.6+ KB


/tmp/ipykernel_33254/4104420128.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offense['timeOnIce20'] = offense['timeOnIce20'].str.replace(':', '.').astype(float)
/tmp/ipykernel_33254/4104420128.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offense.fillna(0.0, inplace=True)


In [52]:
offense

,fullName,timeOnIce20,assists20,goals20,pim20,shots20,games20,hits20
0,P.K. Subban,1504.25,11.0,7.0,79.0,151.0,68.0,88.0
1,Ryan Murray,538.48,7.0,2.0,4.0,32.0,27.0,12.0
2,Damon Severson,1584.26,23.0,8.0,52.0,114.0,69.0,81.0
3,Connor Carrick,417.53,5.0,1.0,17.0,25.0,29.0,27.0
4,Will Butcher,1003.22,17.0,4.0,6.0,62.0,56.0,31.0
...,...,...,...,...,...,...,...,...
471,Alex Tuch,614.04,9.0,8.0,8.0,98.0,42.0,58.0
472,Keegan Kolesar,12.06,0.0,0.0,0.0,1.0,1.0,2.0
473,Nicolas Roy,320.01,5.0,5.0,8.0,42.0,28.0,39.0
474,Nolan Patrick,0.00,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
offense.select_dtypes('number')

,timeOnIce20,assists20,goals20,pim20,shots20,games20,hits20
0,1504.25,11.0,7.0,79.0,151.0,68.0,88.0
1,538.48,7.0,2.0,4.0,32.0,27.0,12.0
2,1584.26,23.0,8.0,52.0,114.0,69.0,81.0
3,417.53,5.0,1.0,17.0,25.0,29.0,27.0
4,1003.22,17.0,4.0,6.0,62.0,56.0,31.0
...,...,...,...,...,...,...,...
471,614.04,9.0,8.0,8.0,98.0,42.0,58.0
472,12.06,0.0,0.0,0.0,1.0,1.0,2.0
473,320.01,5.0,5.0,8.0,42.0,28.0,39.0
474,0.00,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
from dash import html

def discrete_background_color_bins(df, n_bins=5, columns='all'):
    import colorlover
    bounds = [i * (1.0 / n_bins) for i in range(n_bins + 1)]
    if columns == 'all':
        if 'id' in df:
            df_numeric_columns = df.select_dtypes('number').drop(['id'], axis=1)
        else:
            df_numeric_columns = df.select_dtypes('number')
    else:
        df_numeric_columns = df[columns]
    df_max = df_numeric_columns.max()
    df_min = df_numeric_columns.min()
    ranges = [
        ((df_max - df_min) * i) + df_min
        for i in bounds
        ]
    ranges = pd.DataFrame(ranges)
    styles = []
    legend = []
    for column in df_numeric_columns:

        for i in range(1, len(bounds)):
            min_bound = ranges[column][i - 1]
            max_bound = ranges[column][i]
            backgroundColor = colorlover.scales[str(n_bins)]['seq']['Blues'][i - 1]
            color = 'white' if i > len(bounds) / 2. else 'inherit'

            styles.append({
                'if': {
                    'filter_query': (
                        '{{{column}}} >= {min_bound}' +
                        (' && {{{column}}} < {max_bound}' if (i < len(bounds) - 1) else '')
                    ).format(column=column, min_bound=min_bound, max_bound=max_bound),
                    'column_id': column
                },
                'backgroundColor': backgroundColor,
                'color': color
            })
        legend.append(
            html.Div(style={'display': 'inline-block', 'width': '60px'}, children=[
                html.Div(
                    style={
                        'backgroundColor': backgroundColor,
                        'borderLeft': '1px rgb(50, 50, 50) solid',
                        'height': '10px'
                    }
                ),
                html.Small(round(min_bound, 2), style={'paddingLeft': '2px'})
            ])
        )

    return (styles, html.Div(legend, style={'padding': '5px 0 5px 0'}), df_max, df_min, ranges)

(styles, legend, df_max, df_min, ranges) = discrete_background_color_bins(offense)

In [70]:
df_max.to_dict()

{'timeOnIce20': 1846.16,
 'assists20': 67.0,
 'goals20': 48.0,
 'pim20': 122.0,
 'shots20': 318.0,
 'games20': 71.0,
 'hits20': 303.0}

In [77]:
df_min.to_dict()

{'timeOnIce20': 0.0,
 'assists20': 0.0,
 'goals20': 0.0,
 'pim20': 0.0,
 'shots20': 0.0,
 'games20': 0.0,
 'hits20': 0.0}

In [86]:
pd.DataFrame(ranges)['timeOnIce20'][2 - 1]

369.232

In [54]:
cols = [col for col in df.columns if '14' or 'fullName' in col]

In [55]:
df_14 = df[cols]

In [56]:
df_14[df_14['Salary_2014-15'] != 0.0]

,fullName,name,currentAge,birthCity,birthCountry,height,weight,shootsCatches,rosterStatus,timeOnIce20,...,evenTimeOnIcePerGame14,shortHandedTimeOnIcePerGame14,powerPlayTimeOnIcePerGame14,Salary_2014-15,Salary_2015-16,Salary_2016-17,Salary_2017-18,Salary_2018-19,Salary_2019-20,Salary_2020-21
0,P.K. Subban,Defenseman,32,Toronto,CAN,"6' 0""",210,R,Y,1504:25,...,19:17,00:40,04:39,7000000.0,7000000.0,11000000.0,11000000.0,10000000.0,10000000.0,8000000.0
13,Travis Zajac,Center,36,Winnipeg,CAN,"6' 2""",185,R,Y,1179:36,...,15:48,01:59,02:30,5000000.0,6500000.0,6500000.0,6500000.0,6500000.0,5750000.0,1536639.0
45,Claude Giroux,Center,33,Hearst,CAN,"5' 11""",185,R,Y,1309:57,...,15:44,00:56,03:44,10000000.0,9000000.0,9000000.0,9000000.0,9000000.0,8000000.0,7200000.0
61,Jeff Carter,Center,36,London,CAN,"6' 3""",219,R,Y,1016:05,...,14:10,01:30,03:16,6750000.0,7000000.0,7000000.0,6500000.0,5000000.0,3000000.0,448266.0
62,Sidney Crosby,Center,33,Cole Harbour,CAN,"5' 11""",200,L,Y,831:00,...,17:10,00:30,04:17,12000000.0,12000000.0,10900000.0,10900000.0,10000000.0,9000000.0,9600000.0
80,Patrice Bergeron,Center,35,L'Ancienne-Lorette,CAN,"6' 1""",195,R,Y,1142:16,...,14:02,01:57,01:59,8750000.0,8750000.0,8750000.0,8750000.0,6875000.0,4375000.0,4375000.0
199,Alex Ovechkin,Left Wing,35,Moscow,RUS,"6' 3""",238,R,Y,1405:11,...,15:29,00:00,05:02,10000000.0,10000000.0,10000000.0,10000000.0,10000000.0,10000000.0,10000000.0
295,Gabriel Landeskog,Left Wing,28,Stockholm,SWE,"6' 1""",215,L,Y,1081:35,...,16:03,00:12,02:24,4500000.0,4500000.0,5500000.0,6000000.0,6000000.0,6000000.0,6500000.0
307,Duncan Keith,Defenseman,38,Winnipeg,CAN,"6' 1""",192,L,Y,1487:31,...,19:05,02:23,03:10,7600000.0,7500000.0,6000000.0,5000000.0,4500000.0,3500000.0,2650000.0
345,Ryan Getzlaf,Center,36,Regina,CAN,"6' 3""",228,R,Y,1292:10,...,15:28,02:07,03:40,8750000.0,9250000.0,9250000.0,9025000.0,8950000.0,8275000.0,6000000.0
